In [2]:
import pandas as pd
load = pd.read_csv("C:\\Users\\HP\\Downloads\\machinepredictivemaintanence\\predictive_maintenance.csv")
load

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure
...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure


In [3]:
load = load.drop(columns=['UDI','Product ID'],axis=1)


In [4]:
load.isnull().sum()

Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Target                     0
Failure Type               0
dtype: int64

In [5]:
load.describe()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,300.004930,310.005560,1538.776100,39.986910,107.951000,0.033900
std,2.000259,1.483734,179.284096,9.968934,63.654147,0.180981
min,295.300000,305.700000,1168.000000,3.800000,0.000000,0.000000
25%,298.300000,308.800000,1423.000000,33.200000,53.000000,0.000000
50%,300.100000,310.100000,1503.000000,40.100000,108.000000,0.000000
75%,301.500000,311.100000,1612.000000,46.800000,162.000000,0.000000
max,304.500000,313.800000,2886.000000,76.600000,253.000000,1.000000


In [6]:
load['Air temperature [K]'] = load['Air temperature [K]']-273.15
load['Process temperature [K]'] = load['Process temperature [K]']-273.15
load['temperature Difference'] = load['Process temperature [K]'] - load['Air temperature [K]']
load
load.rename(columns={"Air temperature [K]":"Air temperature [C]","process temperature [K]":"process temperature [K]"},inplace=True)

In [7]:
for cols in ['Type','Target','Failure Type']:
  print(load[cols].value_counts())

Type
L    6000
M    2997
H    1003
Name: count, dtype: int64
Target
0    9661
1     339
Name: count, dtype: int64
Failure Type
No Failure                  9652
Heat Dissipation Failure     112
Power Failure                 95
Overstrain Failure            78
Tool Wear Failure             45
Random Failures               18
Name: count, dtype: int64


In [8]:
!pip install category_encoders



In [9]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=['Type','Failure Type'])
load = encoder.fit_transform(load)
load.head(2)

,Type,Air temperature [C],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type,temperature Difference
0,1,24.95,35.45,1551,42.8,0,0,1,10.5
1,2,25.05,35.55,1408,46.3,3,0,1,10.5


In [10]:
from sklearn.preprocessing import LabelEncoder
scaler = LabelEncoder()
load['Failure Type'] = scaler.fit_transform(load['Failure Type'])
load.drop(columns="Target")

,Type,Air temperature [C],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,temperature Difference
0,1,24.95,35.45,1551,42.8,0,0,10.5
1,2,25.05,35.55,1408,46.3,3,0,10.5
2,2,24.95,35.35,1498,49.4,5,0,10.4
3,2,25.05,35.45,1433,39.5,7,0,10.4
4,2,25.05,35.55,1408,40.0,9,0,10.5
...,...,...,...,...,...,...,...,...
9995,1,25.65,35.25,1604,29.5,14,0,9.6
9996,3,25.75,35.25,1632,31.8,17,0,9.5
9997,1,25.85,35.45,1645,33.4,22,0,9.6
9998,3,25.85,35.55,1408,48.5,25,0,9.7


In [11]:
X = load.drop(columns="Failure Type" , axis=1)
y = load["Failure Type"]

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=21)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
y_pred_rf = model.predict(X_test)
model.score(X_train, y_train)
Random_Forest_accuracy = round(accuracy_score(y_pred_rf, y_test) * 100, 2)
print(Random_Forest_accuracy)

99.6


In [14]:
from joblib import dump
dump(model,'./../savedModels/model.joblib')

['./../savedModels/model.joblib']